In [30]:
#DQN. 深層学習とQ学習を組み合わせた手法  経験をたくさんメモリに貯めてあとでランダムに学習
#行動評価関数をNNで表現  
#パッケージのインポート
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque
from tensorflow.compat.v1.losses import huber_loss

In [31]:
#パラメータの準備
NUM_EPISODES=500 #学習するエピソード数
MAX_STEPS=200 #最大ステップ数　1エピソード内の
GAMMA=0.99 #時間割引率 Q学習の
WARMUP=10 #無操作ステップ数 環境をリセットした際の無操作

#探索パラメータ
E_START=1.0 #εの初期値
E_STOP=0.01 #εの最終値
E_DECAY_RATE=0.001 #εの減衰率

#メモリパラメータ 
MEMORY_SIZE=10000 #経験メモリのサイズ
BATCH_SIZE=32 #バッチサイズ

In [32]:
#全結合層を４つ重ねたモdる
#行動評価関数の定義
class QNetwork:
  #初期化
  def __init__(self,state_size,action_size):
    #モデルの作成
    self.model=Sequential()
    self.model.add(Dense(16,activation='relu',input_dim=state_size))
    self.model.add(Dense(16,activation='relu'))
    self.model.add(Dense(16,activation='relu'))
    self.model.add(Dense(action_size,activation='linear'))

    #モデルのコンパイル
    self.model.compile(loss=huber_loss,optimizer=Adam(lr=0.001))
    
    
    

In [33]:
#経験メモリの定義　経験，状態，行動，報酬，次の状態
class Memory():
  #初期化
    def __init__(self,memory_size):
      self.buffer=deque(maxlen=memory_size)

    #経験の追加
    def add(self,experience):
      self.buffer.append(experience)

    #バッチサイズ分の経験をランダムに取得  NNの学習を行う
    def sample(self,batch_size):
      idx=np.random.choice(np.arange(len(self.buffer)),size=batch_size,replace=False)
      return [self.buffer[i] for i in idx]

    #経験メモリのサイズ
    def __len__(self):
      return len(self.buffer)

In [34]:
#環境の作成
env=gym.make('CartPole-v0')
state_size=env.observation_space.shape[0] #行動数
action_size=env.action_space.n #状態数

#main-networkの作成
main_qn=QNetwork(state_size,action_size)

#target-networkの作成
target_qn=QNetwork(state_size,action_size)

#経験メモリの作成
memory=Memory(MEMORY_SIZE)

In [35]:
#学習の開始　環境のリセット　エピソードの反復 
#環境の初期化
state=env.reset()
state=np.reshape(state,[1,state_size])

#エピソード数分のエピソードを繰り返す
total_step=0 #総ステップ数
success_count=0 #成功数
for episode in range(1,NUM_EPISODES+1):
  step=0 #ステップ数

  #target-networkの更新
  target_qn.model.set_weights(main_qn.model.get_weights())

  #1エピソードのループ

  #エピソード完了時のログ表示
  print('エピソード:{}, ステップ数:{},epsilon:{:.4f}'.format(episode,step,epsilon))

  #５回連続成功で学習終了
  if success_count>=5:
    break

  #環境のリセット
  state=env.reset()
  state=np.reshape(state,[1,state_size])

エピソード:1, ステップ数:0,epsilon:0.8205
エピソード:2, ステップ数:0,epsilon:0.8205
エピソード:3, ステップ数:0,epsilon:0.8205
エピソード:4, ステップ数:0,epsilon:0.8205
エピソード:5, ステップ数:0,epsilon:0.8205
エピソード:6, ステップ数:0,epsilon:0.8205
エピソード:7, ステップ数:0,epsilon:0.8205
エピソード:8, ステップ数:0,epsilon:0.8205
エピソード:9, ステップ数:0,epsilon:0.8205
エピソード:10, ステップ数:0,epsilon:0.8205
エピソード:11, ステップ数:0,epsilon:0.8205
エピソード:12, ステップ数:0,epsilon:0.8205
エピソード:13, ステップ数:0,epsilon:0.8205
エピソード:14, ステップ数:0,epsilon:0.8205
エピソード:15, ステップ数:0,epsilon:0.8205
エピソード:16, ステップ数:0,epsilon:0.8205
エピソード:17, ステップ数:0,epsilon:0.8205
エピソード:18, ステップ数:0,epsilon:0.8205
エピソード:19, ステップ数:0,epsilon:0.8205
エピソード:20, ステップ数:0,epsilon:0.8205
エピソード:21, ステップ数:0,epsilon:0.8205
エピソード:22, ステップ数:0,epsilon:0.8205
エピソード:23, ステップ数:0,epsilon:0.8205
エピソード:24, ステップ数:0,epsilon:0.8205
エピソード:25, ステップ数:0,epsilon:0.8205
エピソード:26, ステップ数:0,epsilon:0.8205
エピソード:27, ステップ数:0,epsilon:0.8205
エピソード:28, ステップ数:0,epsilon:0.8205
エピソード:29, ステップ数:0,epsilon:0.8205
エピソード:30, ステップ数:0,epsilon:0.8205
エピソード:31, ステップ数:0,e

In [36]:
#1エピソードのループ
for _ in range(1,MAX_STEPS+1):
  step+=1
  total_step+=1

  #εを減らす
  epsilon=E_STOP+(E_START-E_STOP)*np.exp(-E_DECAY_RATE*total_step)

  #ランダムな行動を選択
  if epsilon>np.random.rand():
    action=env.action_space.sample()
  #行動価値関数で行動を選択
  else:
    action=np.argmax(main_qn.model.predict(state)[0])

    #行動に応じて状態と報酬を得る
    next_state, _, done, _ =env.step(action)
    next_state=np.reshape(next_state,[1,state_size])

    #エピソード完了時
    if done:
      #報酬の指定
      if step>=190:
        success_count+=1
        reward=1
      else:
        success_count=0
        reward=0

      #次の状態に状態なしを代入
      next_state=np.zeros(state.shape)

      #経験の追加
      if step>WARMUP:
        memory.add((state,action,reward,next_state))

      #状態に次の状態を代入
      state=next_state

    #行動価値関数の更新
    if len(memory)>=BATCH_SIZE:
      #NNの入力と出力の準備
      inputs=np.zeros((BATCH_SIZE,4)) #入力(状態)
      targets=np.zeros((BATCH_SIZE,2)) #出力(行動ごとの価値)

      #バッチサイズ分の経験をランダムに取得
      minibatch=memory.sample(BATCH_SIZE)

      #NNの入力と出力の生成
      for i, (state_b,action_b,reward_b,next_state_b) in enumerate(minibatch):
        
        #入力に状態を指定
        inputs[i]=state_b

        #とった行動の価値を計算
        if not(next_state_b==np.zeros(state_b.shape)).all(axis=1):
          target=reward_b+GAMMA*np.amax(target_qn.model.predict(next_state_b)[0])
        else:
          target=reward_b

        #出力に行動ごとの価値を指定
        targets[i]=main.qn.predict(state_b)
        targets[i][action_b]=target #とった行動の価値

      #エピソード完了時
      if done:
        break

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [37]:
#ディスプレイ設定のインストール
!apt-get -qq -y install xvfb freelut3-dev ffmpeg> /dev/null
!pip install pyglet
!pip install pyopengl
!pip install pyvirtualdisplay

#ディスプレイ設定の適用
from pyvirtualdisplay import Display
import os
disp=Display(visible=0,size=(1024,768))
disp.start()
os.environ['DISPLAY']=':' +str(disp.display)+ '.' +str(disp.screen)

E: Unable to locate package freelut3-dev


EasyProcessError: ignored

In [ ]:
#評価
frames=[]  #アニメーションフレーム

#環境のリセット
state=env.reset()
state=np.reshape(state,[1,state_size])

#1エピソードのループ
step=0 #ステップ数
for step in range(1,MAX_STEPS+1):
  step+=1

  #アニメーションフレームの追加
  frames.append(env.render(mode='rgb_array'))

  #最適行動を選択
  action=np.argmax(main_qn.model.predict(state)[0])

  #行動に応じて状態と報酬を得る
  next_state,reward,done, _ =env.step(action)
  next_state=np.reshape(next_state,[1,state_size])

  #エピソード完了
  if done:
    #次の状態に状態なしを代入
    next_state=np.zeros(state.shape)

    break
  else:
    state=next_state
print('ステップ数:{}'.format(step))


In [ ]:
#JSAnimationのインストール
!pip install JSAnimation

#パッケージのインポート
import matplotlib.pyplot as plt
from matplotlib import animation
from JSAnimation.IPython_display import display_animation
from IPython.display import display

#アニメーション再生の定義
def display_frame_as_gif(frame):
  plt.figure(figsize=(frames[0].shape[1]/72.0,frames[0].shaple[0]/72.0),dpi=72)
  patch=plt.imshow(frames[0])
  plt.axis('off')

  #アニメーションの定期処理
  def animate(i):
    patch.set_data(frames[i])

    anim=animation.FuncAnimation(plt.gcf(),animate,frames=len(frames),interval=50)
    display(display_animation(anim,default_node='loop'))

display_frames_as_gif(frames)